In [1]:
# Top 50 US cities by population with their center coordinates (lat, lon)
TOP_50_CITIES = {
    "New York": (40.7128, -74.0060),
    "Los Angeles": (34.0522, -118.2437),
    "Chicago": (41.8781, -87.6298),
    "Houston": (29.7604, -95.3698),
    "Phoenix": (33.4484, -112.0740),
    "Philadelphia": (39.9526, -75.1652),
    "San Antonio": (29.4241, -98.4936),
    "San Diego": (32.7157, -117.1611),
    "Dallas": (32.7767, -96.7970),
    "San Jose": (37.3382, -121.8863),
    "Austin": (30.2672, -97.7431),
    "Jacksonville": (30.3322, -81.6557),
    "Fort Worth": (32.7555, -97.3308),
    "Columbus": (39.9612, -82.9988),
    "Charlotte": (35.2271, -80.8431),
    "San Francisco": (37.7749, -122.4194),
    "Indianapolis": (39.7684, -86.1581),
    "Seattle": (47.6062, -122.3321),
    "Denver": (39.7392, -104.9903),
    "Washington DC": (38.9072, -77.0369),
    "Boston": (42.3601, -71.0589),
    "El Paso": (31.7619, -106.4850),
    "Nashville": (36.1627, -86.7816),
    "Detroit": (42.3314, -83.0458),
    "Oklahoma City": (35.4676, -97.5164),
    "Portland": (45.5152, -122.6784),
    "Las Vegas": (36.1699, -115.1398),
    "Memphis": (35.1495, -90.0490),
    "Louisville": (38.2527, -85.7585),
    "Baltimore": (39.2904, -76.6122),
    "Milwaukee": (43.0389, -87.9065),
    "Albuquerque": (35.0844, -106.6504),
    "Tucson": (32.2226, -110.9747),
    "Fresno": (36.7378, -119.7871),
    "Mesa": (33.4152, -111.8315),
    "Sacramento": (38.5816, -121.4944),
    "Kansas City": (39.0997, -94.5786),
    "Colorado Springs": (38.8339, -104.8214),
    "Omaha": (41.2565, -95.9345),
    "Raleigh": (35.7796, -78.6382),
    "Miami": (25.7617, -80.1918),
    "Long Beach": (33.7701, -118.1937),
    "Virginia Beach": (36.8529, -75.9780),
    "Oakland": (37.8044, -122.2712),
    "Minneapolis": (44.9778, -93.2650),
    "Tulsa": (36.1540, -95.9928),
    "Tampa": (27.9506, -82.4572),
    "Arlington": (32.7357, -97.1081),
    "New Orleans": (29.9511, -90.0715)
}

In [ ]:
# Import relevant components from Sionna RT
import matplotlib.pyplot as plt
import numpy as np
import mitsuba as mi
import warnings
import sys
import os

from sionna.rt import load_scene, Transmitter, Receiver, Transmitter, Camera, PathSolver, RadioMapSolver
from sionna.rt import AntennaArray, PlanarArray, SceneObject, ITURadioMaterial
from sionna.rt.antenna_pattern import antenna_pattern_registry

# Add the src directory to the Python path
sys.path.append(os.path.abspath('../src'))

# Building placement code
from scene_parser import extract_building_info
from tx_placement import TxPlacement
from boresight_pathsolver import create_zone_mask, optimize_boresight_pathsolver
from angle_utils import azimuth_elevation_to_yaw_pitch
from zone_validator import find_valid_zone

# Running the optimizations on 50 scenes
# These scenes make up the 50 most populous cities in the United States
# Experiment Description (Per Scene):
# Map: 1km x 1 km
# Target: 250m x 250m square zones
# Scenarios: Zone placed 200-400m from TX to ensure multipath propagation
# Frequency: 3.67 GHz
# Tx Placement: Most central building with a 5.0 m offset in Z
# UE Z-Height: 1.5 m
# Initial Orientation: Naive Baseline -> Centroid
# Loss function is single objective: Raise Geometric Mean
# Analysis: RadioMapSolver -> Zone Power per 1x1 meter cell (BEFORE and AFTER optimization)
# Choice of LDS: Sobol, Halton, Latin
# Choice of Sampling Method: Rejection, CDT + Turk's 
# Zone Validation: Ensures high-stakes scenarios (-140 <= p10 <= -80 dBm, p90 >= -130 dBm, range >= 15 dB)

# Get list of scene subdirectories and sort them alphabetically
parent_folder = "../scene/scenes"
scene_dirs = sorted([d for d in os.listdir(parent_folder) if os.path.isdir(os.path.join(parent_folder, d))])

# Dictionary to store all results keyed by scene name
results = {}

# RadioMapSolver()
rm_solver = RadioMapSolver()

# Zone validation thresholds for high-stakes scenarios
validation_thresholds = {
    'p10_min_dbm': -160.0,              # Reject zones with 10th percentile < -200 dBm (too weak/dead)
    'p10_max_dbm': -90.0,               # Reject zones with 10th percentile > -80 dBm (too strong)
    'p90_min_dbm': -105.0,              # Reject zones with 90th percentile < -130 dBm (too weak)
    'min_percentile_range_db': 15.0,    # Require at least 15 dB range between p90 and p10
    'median_max_dbm': -60.0             # I don't want a median that's already too healthy
}

print(f"Testing {len(scene_dirs)} scenes")
print(f"Validation thresholds: {validation_thresholds}\n")

for i, scene_name in enumerate(scene_dirs[:10]):    
    print(f"Scene {i+1}/{len(scene_dirs)}: {scene_name}")
    # Build path to scene XML
    scene_xml_path = os.path.join(parent_folder, scene_name, "scene.xml")
    # Load scene
    scene = load_scene(scene_xml_path)
    
    # Set up the scenario
    # Set the operating frequency (n48 band for 5G)
    scene.frequency = 3.7e9  # 3.7 GHz

    # gNB antenna: 3GPP TR 38.901 pattern (AIRSTRAN D 2200)
    gnb_pattern_factory = antenna_pattern_registry.get("tr38901")
    gnb_pattern = gnb_pattern_factory(polarization="V")

    # UE antenna: Isotropic pattern (typical for mobile devices)
    # This will be required for matching the calculations of the RadioMapSolver()
    ue_pattern_factory = antenna_pattern_registry.get("iso")
    # Polarization should also match the transmitter
    ue_pattern = ue_pattern_factory(polarization="V")

    # SISO: Single antenna element at origin [0, 0, 0] for both TX and RX
    single_element = np.array([[0.0, 0.0, 0.0]])  # Shape: (1, 3)

    # Configure antenna arrays
    scene.tx_array = AntennaArray(
        antenna_pattern=gnb_pattern,
        normalized_positions=single_element.T  # Shape: (3, 1)
    )

    scene.rx_array = AntennaArray(
        antenna_pattern=ue_pattern,
        normalized_positions=single_element.T  # Shape: (3, 1)
    )

    # Disable scattering for basic simulation
    for radio_material in scene.radio_materials.values():
        radio_material.scattering_coefficient = 0.0

    # Select building and establish the transmitter
    building_info = extract_building_info(scene_xml_path, verbose=False)

    # Find the most central building (closest to 0,0)
    min_distance = float('inf')
    selected_building_id = None
    
    for building_id, info in building_info.items():
        x_center, y_center = info['center']
        # Calculate Euclidean distance from (0, 0)
        distance = np.sqrt(x_center**2 + y_center**2)
        if distance < min_distance:
            min_distance = distance
            selected_building_id = building_id
    
    print(f"Selected most central building: {selected_building_id} (distance from origin: {min_distance:.2f}m)")

    # TxPlacement will create the transmitter if it doesn't exist and place it on the building
    # Correct parameter order: (scene, tx_name, scene_xml_path, building_id, offset)
    tx_placer = TxPlacement(scene, "gnb", scene_xml_path, selected_building_id, offset=5.0)
    tx_placer.set_rooftop_center()

    # Get reference to the transmitter (already added to scene by TxPlacement)
    tx = tx_placer.tx
    # Convert to flat numpy array instead of nested list
    gnb_position = tx.position.numpy().flatten().tolist()
    
    print(f"gNB placed on building {selected_building_id} at position: {gnb_position}")  

    # Map Data
    # Define the map configuration
    map_config = {
        'center': [0.0, 0.0, 0.0],
        'size': [1000, 1000],
        'cell_size': (1.0, 1.0),
        'ground_height': 0.0,
    }   

    # Find a valid zone using automatic search with validation
    zone_params_template = {
        'width': 250.0,
        'height': 250.0
    }
    
    zone_mask, zone_stats, zone_center, validation_stats, attempts = find_valid_zone(
        scene=scene,
        tx_name="gnb",
        tx_position=gnb_position,
        map_config=map_config,
        scene_xml_path=scene_xml_path,
        zone_params_template=zone_params_template,
        min_distance=50.0,
        max_distance=300.0,
        max_attempts=200,
        validation_kwargs=validation_thresholds,
        verbose=True
    )
    
    # Check if valid zone was found
    if zone_mask is None:
        print(f"✗ Could not find valid zone for {scene_name} after {attempts} attempts - skipping scene\n")
        results[scene_name] = {
            'status': 'failed',
            'reason': 'No valid zone found',
            'attempts': attempts
        }
        print("="*80 + "\n")
        continue
    
    zone_center_x, zone_center_y = zone_center
    zone_params = zone_stats['zone_params']
    
    print(f"✓ Found valid zone after {attempts} attempt(s)")
    print(f"  Zone center: ({zone_center_x:.1f}, {zone_center_y:.1f})")
    print(f"  P10: {validation_stats['p10_power_dbm']:.1f} dBm, P90: {validation_stats['p90_power_dbm']:.1f} dBm")
    print(f"  Percentile range: {validation_stats['percentile_range_db']:.1f} dB")
    
    # Calculate zone distance and angle from TX for logging
    zone_distance_from_tx = np.sqrt((zone_center_x - gnb_position[0])**2 + (zone_center_y - gnb_position[1])**2)
    zone_angle_from_tx = np.arctan2(zone_center_y - gnb_position[1], zone_center_x - gnb_position[0])

    # Run optimization to get initial and best angles
    best_angles, loss_hist, angle_hist, grad_hist, cov_stats, initial_angles = optimize_boresight_pathsolver(
        scene=scene,
        tx_name="gnb",
        map_config=map_config,
        scene_xml_path=scene_xml_path,
        zone_mask=zone_mask,
        zone_params=zone_params,
        zone_stats=zone_stats,
        num_sample_points=50,
        learning_rate=2.0,
        num_iterations=100,
        verbose=True,
        lds="Latin",
        save_radiomap_frames=False,
        frame_save_interval=10,
        output_dir="./frames/"
    )
    
    print(f"\nOptimization complete!")
    print(f"  Initial angles: Azimuth={initial_angles[0]:.1f}°, Elevation={initial_angles[1]:.1f}°")
    print(f"  Best angles:    Azimuth={best_angles[0]:.1f}°, Elevation={best_angles[1]:.1f}°")

    # ===== EVALUATION WITH INITIAL ANGLES (BASELINE) =====
    print(f"\nEvaluating with INITIAL angles...")
    # Set transmitter orientation to initial angles
    yaw_initial, pitch_initial = azimuth_elevation_to_yaw_pitch(initial_angles[0], initial_angles[1])
    tx.orientation = mi.Point3f(float(yaw_initial), float(pitch_initial), 0.0)
    
    # Generate radio map with initial orientation
    rm_initial = rm_solver(
        scene,
        max_depth=5,
        samples_per_tx=int(6e8),
        cell_size=map_config['cell_size'],
        center=map_config['center'],
        orientation=[0, 0, 0],
        size=map_config['size'],
        los=True,
        specular_reflection=True,
        diffuse_reflection=True,
        diffraction=True,
        edge_diffraction=True,
        refraction=False,
        stop_threshold=None,
    )
    
    # Extract zone power for initial configuration
    rss_watts_initial = rm_initial.rss.numpy()[0, :, :]
    signal_strength_dBm_initial = 10.0 * np.log10(rss_watts_initial + 1e-30) + 30.0
    zone_power_initial = signal_strength_dBm_initial[zone_mask == 1.0]
    
    print(f"  Initial mean power: {np.mean(zone_power_initial):.2f} dBm")

    # ===== EVALUATION WITH OPTIMIZED ANGLES =====
    print(f"Evaluating with OPTIMIZED angles...")
    # Set transmitter orientation to best angles
    yaw_best, pitch_best = azimuth_elevation_to_yaw_pitch(best_angles[0], best_angles[1])
    tx.orientation = mi.Point3f(float(yaw_best), float(pitch_best), 0.0)
    
    # Generate radio map with optimized orientation
    rm_optimized = rm_solver(
        scene,
        max_depth=5,
        samples_per_tx=int(6e8),
        cell_size=map_config['cell_size'],
        center=map_config['center'],
        orientation=[0, 0, 0],
        size=map_config['size'],
        los=True,
        specular_reflection=True,
        diffuse_reflection=True,
        diffraction=True,
        edge_diffraction=True,
        refraction=False,
        stop_threshold=None,
    )
    
    # Extract zone power for optimized configuration
    rss_watts_optimized = rm_optimized.rss.numpy()[0, :, :]
    signal_strength_dBm_optimized = 10.0 * np.log10(rss_watts_optimized + 1e-30) + 30.0
    zone_power_optimized = signal_strength_dBm_optimized[zone_mask == 1.0]
    
    print(f"  Optimized mean power: {np.mean(zone_power_optimized):.2f} dBm")
    print(f"  10th Percentile Improvement: {np.percentile(zone_power_optimized, 10) - np.percentile(zone_power_initial, 10):.2f} dB")
    print(f"  Median Improvement: {np.median(zone_power_optimized) - np.median(zone_power_initial):.2f} dB")

    # Save all relevant data to dictionary
    results[scene_name] = {
        'status': 'success',
        'scene_xml_path': scene_xml_path,
        'map_config': map_config,
        'initial_angles': initial_angles,
        'best_angles': best_angles,
        'loss_hist': loss_hist,
        'angle_hist': angle_hist,
        'grad_hist': grad_hist,
        'cov_stats': cov_stats,
        'tx_building_id': selected_building_id,
        'tx_position': gnb_position,
        'zone_center': [zone_center_x, zone_center_y],
        'zone_distance_from_tx': zone_distance_from_tx,
        'zone_angle_from_tx': zone_angle_from_tx,
        'zone_attempts': attempts,
        'validation_stats': validation_stats,
        'zone_params': zone_params,
        'zone_stats': zone_stats,
        'zone_power_initial': zone_power_initial,
        'zone_power_optimized': zone_power_optimized
    }

    print(f"\nResults saved for {scene_name}")
    print("="*80 + "\n")

Testing 51 scenes
Validation thresholds: {'p10_min_dbm': -160.0, 'p10_max_dbm': -90.0, 'p90_min_dbm': -105.0, 'min_percentile_range_db': 15.0, 'median_max_dbm': -60.0}

Scene 1/51: Duke
2026-02-03 17:04:34 WARN  [HDRFilm] Monochrome mode enabled, setting film output pixel format to 'luminance' (was rgb).
Selected most central building: 37 (distance from origin: 34.02m)
gNB placed on building 37 at position: [-23.77073860168457, -24.334800720214844, 15.5]

SEARCHING FOR VALID ZONE
Distance range: 50m - 300m from TX
Max attempts: 200

Attempt 1/200:
  Zone placed at 78.0m, 128.5° from TX

ZONE VALIDATION
Tx Position: [-23.77073860168457, -24.334800720214844, 15.5]
look_at_position: [-72.26773071289062, 36.70884323120117, 1.5]
TX Position: (-23.8, -24.3, 15.5)
Zone Center: (-72.3, 36.7, 1.5)
Naive Orientation: Az=128.5°, El=-10.2°

Running quick coverage check...

Coverage Statistics:
  Min Power:       -270.00 dBm
  10th %ile:        -70.80 dBm
  Median Power:     -55.15 dBm
  Mean Power

KeyboardInterrupt: 

In [ ]:
# Post-processing analysis: Compare INITIAL vs OPTIMIZED performance
# Calculate improvement statistics across all successful scenes

print("="*80)
print("COVERAGE IMPROVEMENT ANALYSIS")
print("="*80)

# Filter results to only include successful optimizations
successful_results = {k: v for k, v in results.items() if v.get('status') == 'success'}
failed_results = {k: v for k, v in results.items() if v.get('status') == 'failed'}

print(f"\nTotal scenes: {len(results)}")
print(f"Successful optimizations: {len(successful_results)}")
print(f"Failed to find valid zone: {len(failed_results)}")

if failed_results:
    print("\nFailed scenes:")
    for scene_name, data in failed_results.items():
        print(f"  - {scene_name}: {data.get('reason')} ({data.get('attempts')} attempts)")

print("\n" + "="*80)

if len(successful_results) == 0:
    print("No successful optimizations to analyze!")
    print("="*80)
else:
    # ===== Per-Scene Statistics (Initial vs Optimized) =====
    print("\nCalculating per-zone statistics (Initial vs Optimized)...")
    print("="*80)

    improvement_summary = []

    for scene_name, data in successful_results.items():
        zone_power_initial = data['zone_power_initial']
        zone_power_optimized = data['zone_power_optimized']
        
        # Calculate statistics for INITIAL configuration
        stats_initial = {
            'mean_power': np.mean(zone_power_initial),
            'median_power': np.median(zone_power_initial),
            'percentile_10': np.percentile(zone_power_initial, 10),
            'percentile_90': np.percentile(zone_power_initial, 90),
            'std_dev': np.std(zone_power_initial),
            'num_cells': len(zone_power_initial)
        }
        
        # Calculate statistics for OPTIMIZED configuration
        stats_optimized = {
            'mean_power': np.mean(zone_power_optimized),
            'median_power': np.median(zone_power_optimized),
            'percentile_10': np.percentile(zone_power_optimized, 10),
            'percentile_90': np.percentile(zone_power_optimized, 90),
            'std_dev': np.std(zone_power_optimized),
            'num_cells': len(zone_power_optimized)
        }
        
        # Calculate improvements
        improvement = {
            'scene_name': scene_name,
            'mean_improvement_dB': stats_optimized['mean_power'] - stats_initial['mean_power'],
            'median_improvement_dB': stats_optimized['median_power'] - stats_initial['median_power'],
            'p10_improvement_dB': stats_optimized['percentile_10'] - stats_initial['percentile_10'],
            'p90_improvement_dB': stats_optimized['percentile_90'] - stats_initial['percentile_90'],
            'std_dev_change_dB': stats_optimized['std_dev'] - stats_initial['std_dev']
        }
        improvement_summary.append(improvement)
        
        # Store statistics back in results dictionary
        results[scene_name]['stats_initial'] = stats_initial
        results[scene_name]['stats_optimized'] = stats_optimized
        results[scene_name]['improvement'] = improvement
        
        # Print per-scene summary
        print(f"\n{scene_name}:")
        print(f"  INITIAL  - Mean: {stats_initial['mean_power']:>7.2f} dBm, Median: {stats_initial['median_power']:>7.2f} dBm")
        print(f"  OPTIMIZED- Mean: {stats_optimized['mean_power']:>7.2f} dBm, Median: {stats_optimized['median_power']:>7.2f} dBm")
        print(f"  IMPROVEMENT: {improvement['mean_improvement_dB']:>+6.2f} dB (mean), {improvement['median_improvement_dB']:>+6.2f} dB (median)")

    print("\n" + "="*80)
    print("Per-zone statistics complete!")
    print("="*80)

    # ===== Cumulative Improvement Analysis =====
    print(f"\n\nCUMULATIVE IMPROVEMENT ACROSS {len(successful_results)} SUCCESSFUL SCENES")
    print("="*80)

    # Calculate average improvements across all scenes
    mean_improvements = [imp['mean_improvement_dB'] for imp in improvement_summary]
    median_improvements = [imp['median_improvement_dB'] for imp in improvement_summary]
    p10_improvements = [imp['p10_improvement_dB'] for imp in improvement_summary]
    p90_improvements = [imp['p90_improvement_dB'] for imp in improvement_summary]

    print(f"\nMean Power Improvement:")
    print(f"  Average improvement:  {np.mean(mean_improvements):>+7.2f} dB")
    print(f"  Std dev:              {np.std(mean_improvements):>8.2f} dB")
    print(f"  Min improvement:      {np.min(mean_improvements):>+7.2f} dB ({improvement_summary[np.argmin(mean_improvements)]['scene_name']})")
    print(f"  Max improvement:      {np.max(mean_improvements):>+7.2f} dB ({improvement_summary[np.argmax(mean_improvements)]['scene_name']})")
    print(f"  Scenes improved:      {sum(1 for x in mean_improvements if x > 0)}/{len(successful_results)}")

    print(f"\nMedian Power Improvement:")
    print(f"  Average improvement:  {np.mean(median_improvements):>+7.2f} dB")
    print(f"  Std dev:              {np.std(median_improvements):>8.2f} dB")
    print(f"  Min improvement:      {np.min(median_improvements):>+7.2f} dB ({improvement_summary[np.argmin(median_improvements)]['scene_name']})")
    print(f"  Max improvement:      {np.max(median_improvements):>+7.2f} dB ({improvement_summary[np.argmax(median_improvements)]['scene_name']})")

    print(f"\n10th Percentile Improvement:")
    print(f"  Average improvement:  {np.mean(p10_improvements):>+7.2f} dB")

    print(f"\n90th Percentile Improvement:")
    print(f"  Average improvement:  {np.mean(p90_improvements):>+7.2f} dB")

    # ===== Aggregate Statistics Across All Cells =====
    print(f"\n\nAGGREGATE STATISTICS (All cells from {len(successful_results)} successful scenes)")
    print("="*80)

    # Collect all zone power values
    all_zone_power_initial = np.concatenate([data['zone_power_initial'] for data in successful_results.values()])
    all_zone_power_optimized = np.concatenate([data['zone_power_optimized'] for data in successful_results.values()])

    aggregate_initial = {
        'mean_power': np.mean(all_zone_power_initial),
        'median_power': np.median(all_zone_power_initial),
        'percentile_10': np.percentile(all_zone_power_initial, 10),
        'percentile_90': np.percentile(all_zone_power_initial, 90),
        'std_dev': np.std(all_zone_power_initial),
        'total_cells': len(all_zone_power_initial)
    }

    aggregate_optimized = {
        'mean_power': np.mean(all_zone_power_optimized),
        'median_power': np.median(all_zone_power_optimized),
        'percentile_10': np.percentile(all_zone_power_optimized, 10),
        'percentile_90': np.percentile(all_zone_power_optimized, 90),
        'std_dev': np.std(all_zone_power_optimized),
        'total_cells': len(all_zone_power_optimized)
    }

    print(f"\nINITIAL Configuration:")
    print(f"  Mean power:       {aggregate_initial['mean_power']:>8.2f} dBm")
    print(f"  Median power:     {aggregate_initial['median_power']:>8.2f} dBm")
    print(f"  10th percentile:  {aggregate_initial['percentile_10']:>8.2f} dBm")
    print(f"  90th percentile:  {aggregate_initial['percentile_90']:>8.2f} dBm")
    print(f"  Std deviation:    {aggregate_initial['std_dev']:>8.2f} dB")
    print(f"  Total cells:      {aggregate_initial['total_cells']:,}")

    print(f"\nOPTIMIZED Configuration:")
    print(f"  Mean power:       {aggregate_optimized['mean_power']:>8.2f} dBm")
    print(f"  Median power:     {aggregate_optimized['median_power']:>8.2f} dBm")
    print(f"  10th percentile:  {aggregate_optimized['percentile_10']:>8.2f} dBm")
    print(f"  90th percentile:  {aggregate_optimized['percentile_90']:>8.2f} dBm")
    print(f"  Std deviation:    {aggregate_optimized['std_dev']:>8.2f} dB")
    print(f"  Total cells:      {aggregate_optimized['total_cells']:,}")

    print(f"\nAGGREGATE IMPROVEMENT:")
    print(f"  Mean power:       {aggregate_optimized['mean_power'] - aggregate_initial['mean_power']:>+8.2f} dB")
    print(f"  Median power:     {aggregate_optimized['median_power'] - aggregate_initial['median_power']:>+8.2f} dB")
    print(f"  10th percentile:  {aggregate_optimized['percentile_10'] - aggregate_initial['percentile_10']:>+8.2f} dB")
    print(f"  90th percentile:  {aggregate_optimized['percentile_90'] - aggregate_initial['percentile_90']:>+8.2f} dB")

    # ===== Validation Statistics =====
    print(f"\n\nZONE VALIDATION STATISTICS")
    print("="*80)
    
    zone_attempts = [data['zone_attempts'] for data in successful_results.values()]
    percentile_ranges = [data['validation_stats']['percentile_range_db'] for data in successful_results.values()]
    p10_values = [data['validation_stats']['p10_power_dbm'] for data in successful_results.values()]
    p90_values = [data['validation_stats']['p90_power_dbm'] for data in successful_results.values()]
    
    print(f"\nZone Search Attempts:")
    print(f"  Mean attempts:    {np.mean(zone_attempts):.1f}")
    print(f"  Max attempts:     {np.max(zone_attempts)}")
    print(f"  Min attempts:     {np.min(zone_attempts)}")
    
    print(f"\nValidated Zone Characteristics:")
    print(f"  P10 (mean):           {np.mean(p10_values):.1f} dBm")
    print(f"  P10 (min):            {np.min(p10_values):.1f} dBm")
    print(f"  P10 (max):            {np.max(p10_values):.1f} dBm")
    print(f"  P90 (mean):           {np.mean(p90_values):.1f} dBm")
    print(f"  P90 (min):            {np.min(p90_values):.1f} dBm")
    print(f"  P90 (max):            {np.max(p90_values):.1f} dBm")
    print(f"  Percentile range (mean):  {np.mean(percentile_ranges):.1f} dB")
    print(f"  Percentile range (min):   {np.min(percentile_ranges):.1f} dB")
    print(f"  Percentile range (max):   {np.max(percentile_ranges):.1f} dB")

    # ===== CDF Comparison Plot =====
    print("\n\nGenerating CDF Comparison (Initial vs Optimized)")
    print("="*80)

    # Sort power values for CDF
    sorted_initial = np.sort(all_zone_power_initial)
    sorted_optimized = np.sort(all_zone_power_optimized)
    cdf = np.arange(1, len(sorted_initial) + 1) / len(sorted_initial)

    # Create comparison plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

    # CDF comparison
    ax1.plot(sorted_initial, cdf * 100, linewidth=2, label='Initial (Baseline)', alpha=0.7)
    ax1.plot(sorted_optimized, cdf * 100, linewidth=2, label='Optimized', alpha=0.7)
    ax1.axhline(y=50, color='gray', linestyle='--', alpha=0.3)
    ax1.axhline(y=10, color='red', linestyle='--', alpha=0.3)
    ax1.axhline(y=90, color='green', linestyle='--', alpha=0.3)
    ax1.set_xlabel('Signal Strength (dBm)', fontsize=12)
    ax1.set_ylabel('Cumulative Probability (%)', fontsize=12)
    ax1.set_title(f'CDF Comparison: Initial vs Optimized\n({len(successful_results)} Validated Scenes)', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.legend(loc='best', fontsize=11)

    # Improvement histogram
    ax2.hist(mean_improvements, bins=20, edgecolor='black', alpha=0.7)
    ax2.axvline(x=np.mean(mean_improvements), color='red', linestyle='--', linewidth=2, 
                label=f'Mean: {np.mean(mean_improvements):+.2f} dB')
    ax2.axvline(x=0, color='gray', linestyle='-', linewidth=1, alpha=0.5)
    ax2.set_xlabel('Mean Power Improvement (dB)', fontsize=12)
    ax2.set_ylabel('Number of Scenes', fontsize=12)
    ax2.set_title(f'Distribution of Improvements Across {len(successful_results)} Scenes', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3, axis='y')
    ax2.legend(fontsize=11)

    plt.tight_layout()
    plt.show()

    print(f"\nAnalysis complete!")
    print(f"Median coverage in target zones improved by an average of {np.mean(median_improvements):+.2f} dB across {len(successful_results)} validated scenes.")
    print("="*80)